In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_model_optimization as tfmot

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Define a deep neural network with 4 hidden layers and 3 dropout layers
def create_model():
    model = keras.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Compile and train the base model
model = create_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Prune the model
# Define pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.2, final_sparsity=0.8, begin_step=0, end_step=1000
    )
}

# Apply pruning
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(create_model(), **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# **Add the UpdatePruningStep callback**
pruning_callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

# Train the model
pruned_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test),
                 callbacks=pruning_callbacks)

# Convert pruned model to a TensorFlow Lite model for quantization
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the model sizes
with open("pruned_quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

import os
original_model_size = os.path.getsize("pruned_quantized_model.tflite") / 1024  # KB
print(f"Quantized Pruned Model Size: {original_model_size:.2f} KB")


Epoch 1/10
1563/1563 [==============================] - 25s 15ms/step - loss: 2.0620 - accuracy: 0.2189 - val_loss: 1.8846 - val_accuracy: 0.3143
Epoch 2/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.8936 - accuracy: 0.3040 - val_loss: 1.8381 - val_accuracy: 0.3137
Epoch 3/10
1563/1563 [==============================] - 23s 15ms/step - loss: 1.8477 - accuracy: 0.3260 - val_loss: 1.7464 - val_accuracy: 0.3720
Epoch 4/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.8159 - accuracy: 0.3380 - val_loss: 1.7349 - val_accuracy: 0.3750
Epoch 5/10
1563/1563 [==============================] - 24s 16ms/step - loss: 1.7957 - accuracy: 0.3450 - val_loss: 1.7202 - val_accuracy: 0.3874
Epoch 6/10
1563/1563 [==============================] - 23s 15ms/step - loss: 1.7787 - accuracy: 0.3540 - val_loss: 1.6947 - val_accuracy: 0.4075
Epoch 7/10
1563/1563 [==============================] - 23s 15ms/step - loss: 1.7584 - accuracy: 0.3617 - val_loss: 1.6789 -

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp0995gjog\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp0995gjog\assets


Quantized Pruned Model Size: 13650.15 KB
